### SI 670: Final Project Regression Model

#### Step 0. Import necessary libraries/packages and dataset

In [21]:
# Import libraries/packages
# Standard Python viz
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# NFL library
import nfl_data_py as nfl

In [22]:
# Import NFL dataset
df = nfl.import_pbp_data([2022, 2023, 2024])
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Print dimensions of the dataframe
print(df.shape)

2022 done.
2023 done.
2024 done.
Downcasting floats.
(148591, 397)


In [11]:
# Define our necessary play types
fantasy_play_types = ['pass', 'run', 'qb_kneel']

# Define the play types to exclude from future consideration
exclude_play_types = [
    'no_play',       # Penalties/timeouts
    'qb_spike',      # QB stopping clock, no fantasy impact
    'field_goal',    # Kicker-specific
    'extra_point',   # Kicker-specific
    'punt',          # Defense/special teams
    'kickoff',       # Defense/special teams
]

# Filter DataFrame to include only fantasy_play_types
df_fantasy_plays = df[
    df['play_type'].isin(fantasy_play_types) & 
    ~df['play_type'].isin(exclude_play_types)
].copy()

print(f"Original plays: {len(df)}")
print(f"Plays retained for skill features: {len(df_fantasy_plays)}")

Original plays: 148591
Plays retained for skill features: 107413


In [40]:
mdl_cols = [
    # Identifiers and Context
    'game_id', 'play_id', 'season', 'week', 'posteam', 'defteam', 'home_team', 'away_team',
    
    # Player Identifiers
    'passer_player_id', 'passer', 'rusher_player_id', 'rusher', 'receiver_player_id', 'receiver', 'fumbled_1_player_id', 'fumbled_2_player_id',
    
    # Play Type and Situational Metrics
    'play_type', 'down', 'ydstogo', 'yardline_100', 'shotgun', 'no_huddle', 
    
    # Passing metrics
    'passing_yards', 'pass_touchdown', 'pass_attempt', 'complete_pass',
    
    # Rushing metrics
    'rushing_yards', 'rush_touchdown', 'rush_attempt',

    # Receiving metrics
    'receiving_yards', 'yards_after_catch',

    # Penalty/turnover metrics
    'penalty_yards', 'interception', 'fumble_lost',

    # Miscellaneous metrics
    'yards_gained', 'epa', 'cpoe', 'td_prob'
]

# Filter DataFrame for retained EDA columns
df_mdl0 = df_fantasy_plays[mdl_cols].copy()
print(f"Final DataFrame shape for feature engineering/modeling: {df_mdl0.shape}")

Final DataFrame shape for feature engineering/modeling: (107413, 38)


#### Step 1. Feature Engineering

1. Define fantasy score rules and target variable.

In [ ]:

# Set scoring rules (Half-PPR for now)
scoring_rules = {
    'pass_yd': 0.04, 'pass_td': 4, 
    'rush_yd': 0.1, 'rush_td': 6, 
    'rec_yd': 0.1, 'rec_td': 6, 
    'rec': 0.5, # Half-PPR point per reception
    'int': -2, 'fumble_lost': -2,
    'qb_kneel_yd': -0.1
}

# Logic to calculate fantasy points for each play
def calculate_fantasy_points_per_play(df, scoring_rules):
    """Calculates all fantasy points earned on a single play based on half-PPR rules."""
    
    # Passing Points (QB)
    df['fp_pass'] = (df['passing_yards'].fillna(0) * scoring_rules['pass_yd']) + \
                    (df['pass_touchdown'].fillna(0) * scoring_rules['pass_td'])
                                
    # Rushing Points (RB/QB)
    df['fp_rush'] = (df['rushing_yards'].fillna(0) * scoring_rules['rush_yd']) + \
                    (df['rush_touchdown'].fillna(0) * scoring_rules['rush_td'])
                                
    # Receiving Points (WR/TE/RB)
    df['fp_rec'] = (df['receiving_yards'].fillna(0) * scoring_rules['rec_yd']) + \
                   (df['pass_touchdown'].fillna(0) * scoring_rules['rec_td']) + \
                   (df['complete_pass'].fillna(0) * scoring_rules['rec'])
    
    # Interception (for primary passer)
    df['fp_int'] = df['interception'].fillna(0) * scoring_rules['int']

    # Fumble Lost (for primary ball carrier/or receiver)
    df['fp_fumble_lost'] = df['fumble_lost'].fillna(0) * scoring_rules['fumble_lost']

    # QB Kneel Adjustment: Must be included in the target variable
    df['fp_kneel'] = np.where(df['play_type'] == 'qb_kneel', 
                              df['yards_gained'].fillna(0) * scoring_rules['qb_kneel_yd'], 0)
    
    return df

# Apply the calculation to the full DataFrame
df_mdl1 = calculate_fantasy_points_per_play(df_mdl0, scoring_rules)

Since the fantasy points for each play have been calculuated, we need to generate a point total for each week for each unique player_id in our dataset

In [ ]:
# Calculate weekly fantasy points for each player and aggregate
def calculate_weekly_fantasy_points_final(df):
    """
    Calculates the total weekly fantasy points (Y) for each player by stacking
    contributions from passing, rushing, receiving, as well as any turnovers that occur.
    """
    
    # Calculate the total points for each respective player id
    
    # Total passing points: passing yards/td + interceptions + qb kneels
    df['fp_pass_total'] = df['fp_pass'].fillna(0) + \
                          df['fp_int'].fillna(0) + \
                          df['fp_kneel'].fillna(0)
    
    # Total rushing points: rushing yards/td
    df['fp_rush_total'] = df['fp_rush'].fillna(0)
    
    # Total receiving points: receiving yards/td + receptions for half point PPR
    df['fp_rec_total'] = df['fp_rec'].fillna(0)
    
    # Fumbles attributed to only the fumbled_1_player_id (rare cases where too, but going to ignore)
    df['fp_fumble_1_total'] = df['fp_fumble_lost'].fillna(0)
    
    
    # Specify ids for relevant fantasy time measureables
    id_vars = ['season', 'week']
    
    # Create directory to link point totals to the respective player ids
    contributions_map = {
        'passer_player_id': 'fp_pass_total',
        'rusher_player_id': 'fp_rush_total',
        'receiver_player_id': 'fp_rec_total',
        'fumbled_1_player_id': 'fp_fumble_1_total',
    }

    # Iterate through player ids to generate totals and store results under one player id column
    contributions = []
    for id_col, point_col in contributions_map.items():
        temp_df = df.rename(columns={id_col: 'player_id', point_col: 'points'})
        contributions.append(temp_df[id_vars + ['player_id', 'points']])

    # Stack contributions and drop any missing player ids
    df_all_points = pd.concat(contributions, ignore_index=True)
    df_all_points.dropna(subset=['player_id'], inplace=True)
    
    # Lastly, group by week and season for each respective player id and return that table
    df_target_Y = df_all_points.groupby(id_vars + ['player_id'])['points'].sum().reset_index()
    
    return df_target_Y.rename(columns={'points': 'Y_target_points'})

Now we need to transform the play_by_play for our features into a long table that is aggregated by week/season

In [ ]:
# Define how each feature column should be aggregated per player/week
feature_agg_rules = {
    'passing_yards': 'sum',
    'rushing_yards': 'sum',
    'receiving_yards': 'sum',
    'pass_touchdown': 'sum',
    'rush_touchdown': 'sum',
    'interception': 'sum',
    'epa': 'mean',
    'cpoe': 'mean',
}

# Define columns for time period aggregation and create feature column names from feature_agg_rules dictionary
id_vars = ['season', 'week'] 
feature_cols = list(feature_agg_rules.keys())

# Select only necessary columns from df_mdl1 dataframe: ids, feature columns and ids from original dataframe
df_select = df_mdl1[id_vars + feature_cols + ['passer_player_id', 'rusher_player_id', 'receiver_player_id']].copy()

# Reshape data using melt function to create a table that has multiple rows per play...one for each player
df_X_long = pd.melt(
    df_select,
    id_vars=id_vars + feature_cols,
    value_vars=['passer_player_id', 'rusher_player_id', 'receiver_player_id'],
    var_name='role_type',
    value_name='player_id'
)
df_X_long.dropna(subset=['player_id'], inplace=True)

# Aggregate features on a weekly level by player_id
df_features_X = df_X_long.groupby(['season', 'week', 'player_id']).agg(feature_agg_rules).reset_index()

# Determine how many total plays to determine how many plays the player was involved in for a week
df_counts = df_X_long.groupby(['season', 'week', 'player_id']).size().reset_index(name='total_plays_involved')

# Merge counts with features
df_features_X = pd.merge(df_features_X, df_counts, on=['season', 'week', 'player_id'], how='left')

print("Aggregated Feature DataFrame (X) - Fixed:")
print(df_features_X.head())

Aggregated Feature DataFrame (X) - Fixed:
   season  week   player_id  passing_yards  rushing_yards  receiving_yards  \
0    2022     1  00-0019596          212.0           -1.0            212.0   
1    2022     1  00-0023459          195.0           -1.0            195.0   
2    2022     1  00-0026143          352.0           12.0            352.0   
3    2022     1  00-0026158          309.0            0.0            309.0   
4    2022     1  00-0026498          240.0            2.0            240.0   

   pass_touchdown  rush_touchdown  interception       epa      cpoe  \
0             1.0             0.0           1.0 -0.012462  3.290235   
1             0.0             0.0           1.0 -0.298142 -2.784771   
2             1.0             0.0           1.0 -0.025235 -4.770667   
3             1.0             0.0           1.0 -0.208456 -4.062917   
4             1.0             0.0           3.0 -0.338503 -0.592032   

   total_plays_involved  
0                    31  
1         

Now, we need to merge our features with the target variable and create time series features based on lagged and rolling averages to be used as predictive inputs.

In [ ]:
# Merge our final set of features with the target variable
df_final = pd.merge(
    df_features_X, 
    df_target_Y,    
    on=['season', 'week', 'player_id'],
    how='left'
)

# If any instances where player didn't have points, set score to zero
df_final['Y_target_points'] = df_final['Y_target_points'].fillna(0)

# Sort data by week to properly create time based features
df_final.sort_values(by=['player_id', 'season', 'week'], inplace=True)

# Last week's points (lagged values)
df_final['Y_lag_1'] = df_final.groupby('player_id')['Y_target_points'].shift(1)

# 3 week rolling average
df_final['Y_roll_avg_3'] = df_final.groupby('player_id')['Y_target_points'].transform(
    lambda x: x.shift(1).rolling(window=3, min_periods=1).mean()
)

# Career average up to that week
df_final['Y_cum_avg'] = df_final.groupby('player_id')['Y_target_points'].transform(
    lambda x: x.shift(1).expanding(min_periods=1).mean()
)

# Any time features that are null, set to zero
time_features = ['Y_lag_1', 'Y_roll_avg_3', 'Y_cum_avg']
df_final[time_features] = df_final[time_features].fillna(0)

print("\nFinal Master DataFrame (df_final) is ready for ML modeling:")
print(df_final.head())


Final Master DataFrame (df_master) is ready for ML modeling:
      season  week   player_id  passing_yards  rushing_yards  receiving_yards  \
0       2022     1  00-0019596          212.0           -1.0            212.0   
318     2022     2  00-0019596          190.0           -2.0            190.0   
634     2022     3  00-0019596          271.0           -1.0            271.0   
943     2022     4  00-0019596          385.0            0.0            371.0   
1254    2022     5  00-0019596          351.0           -3.0            351.0   

      pass_touchdown  rush_touchdown  interception       epa       cpoe  \
0                1.0             0.0           1.0 -0.012462   3.290235   
318              1.0             0.0           0.0 -0.123334 -13.116215   
634              1.0             0.0           0.0 -0.165223   4.887816   
943              3.0             0.0           0.0  0.179459   5.982167   
1254             1.0             0.0           0.0  0.191188   0.707572   



In [61]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error


# Define features and target column
id_cols = ['season', 'week', 'player_id']
target_col = 'Y_target_points'

X = df_final.drop(columns=id_cols + [target_col])
y = df_final[target_col]
X = X.fillna(0) # Fill in any nulls just in case


# Chronological split here because we need to preserve past performance
split_point = int(len(X) * 0.90)

X_train = X.iloc[:split_point]
X_test = X.iloc[split_point:]
y_train = y.iloc[:split_point]
y_test = y.iloc[split_point:]

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Quantile model train for 10%, median and 90% quantiles
quantiles = [0.10, 0.50, 0.90]
models = {}
y_preds = {}

for q in quantiles:
    model = HistGradientBoostingRegressor(
        loss='quantile',
        quantile=q,
        max_iter=500,
        learning_rate=0.05,
        max_depth=6,
        random_state=42
    )
    model.fit(X_train_scaled, y_train)
    models[q] = model
    y_preds[q] = model.predict(X_test_scaled)
    
# Store predictions in single dataframe
df_predictions = pd.DataFrame({
    'y_test': y_test.values,
    'pred_10': y_preds[0.10],
    'pred_50': y_preds[0.50],
    'pred_90': y_preds[0.90]
}, index=y_test.index)


# Model evaluation

# Metric: Pinball Loss function for evaluation
def pinball_loss(y_true, y_pred, quantile):
    """Calculates the Pinball Loss (Quantile Loss). Lower is better."""
    error = y_true - y_pred
    loss = np.where(error >= 0, quantile * error, (1 - quantile) * (-error))
    return np.mean(loss)

# Metric: Mean Absolute Error (MAE)
mae = mean_absolute_error(df_predictions['y_test'], df_predictions['pred_50'])
print(f"Mean Absolute Error (MAE) for Median Prediction: {mae:.2f} points")


# Metric: Prediction Interval Coverage Probability (PICP) - using 0.80 because 0.90 - 0.10
PI_LEVEL = 0.80 
lower_bound = df_predictions['pred_10']
upper_bound = df_predictions['pred_90']

is_covered = (df_predictions['y_test'] >= lower_bound) & \
             (df_predictions['y_test'] <= upper_bound)
             
picp = is_covered.mean() * 100

print(f"\nPrediction Interval Coverage Probability (PICP) for {int(PI_LEVEL*100)}% PI:")
print(f"Expected Coverage: {int(PI_LEVEL*100)}%")
print(f"Actual Coverage: {picp:.2f}%")
print("Note: If Actual Coverage is close to Expected Coverage, the model accurately assesses risk.\n")


# Metric: Average Pinball Loss for Floor (Risk Assessment)
pinball_loss_10 = pinball_loss(df_predictions['y_test'], df_predictions['pred_10'], 0.10)
print(f"Pinball Loss for 10th Quantile (Floor): {pinball_loss_10:.4f}")
print("Note: Lower score means the model better predicts a player's safe 'floor'.")

# Show example of prediction interval for for first player
print("\nExample Prediction Interval")
example_row = df_predictions.iloc[0]
print(f"Actual Points: {example_row['y_test']:.2f}")
print(f"Predicted Median (50%): {example_row['pred_50']:.2f}")
print(f"Predicted 80% Interval: [{example_row['pred_10']:.2f} to {example_row['pred_90']:.2f}]")

Mean Absolute Error (MAE) for Median Prediction: 1.61 points

Prediction Interval Coverage Probability (PICP) for 80% PI:
Expected Coverage: 80%
Actual Coverage: 72.47%
Note: If Actual Coverage is close to Expected Coverage, the model accurately assesses risk.

Pinball Loss for 10th Quantile (Floor): 0.2443
Note: Lower score means the model better predicts a player's safe 'floor'.

Example Prediction Interval
Actual Points: 6.80
Predicted Median (50%): 6.83
Predicted 80% Interval: [1.30 to 7.23]
